In [ ]:
# General Dependencies
import timeit, os
import numpy as np
import scipy.io as io

from denoise import temporal

# Preprocessing Dependencies
from trefide.utils import psd_noise_estimate

# PMD Model Dependencies
from trefide.pmd import batch_decompose, batch_recompose, overlapping_batch_decompose, overlapping_batch_recompose

# Plot & Video Dependencies
import mpdf_plot
import matplotlib.pyplot as plt

# Plots
from trefide.plot import pixelwise_ranks
from trefide.extras.util_plot import comparison_plot
from trefide.video import write_mpl, play_cv2

%load_ext autoreload
%autoreload 2

In [ ]:
dx=4
ext = "../trefide"

In [ ]:
# Set Data Information
dataset = "Endoscope"
filename = os.path.join(ext, "data", "pc_blood_vessel_10Hz.npz")
block_height = 16
block_width = 16

In [ ]:
mov = np.load(filename)['data'].astype(np.float64).copy()
fov_height, fov_width, num_frames = mov.shape
num_pixels = fov_height * fov_width

In [ ]:
# Remove Any Nan Pixels
mov[np.isnan(mov)] = 0

# Remove Pixel-wise Mean
mov -= np.mean(mov, axis=-1)[:,:,None]

# Normalize By Pixelwise Standard Deviation
pixelwise_std = np.reshape(np.sqrt(psd_noise_estimate(np.reshape(mov, (num_pixels, num_frames)))),
                           (fov_height, fov_width, 1))
mov /= pixelwise_std
mov[np.isnan(mov)] = 0

In [ ]:
rank_background = 5
max_iters_main = 10
max_iters_init = 40
consec_failures = 3
d_sub=2
t_sub=2
tol = 5e-3
spatial_components_bg,\
temporal_components_bg,\
block_ranks,\
block_indices = batch_decompose(fov_height, fov_width, num_frames,
                                mov, fov_height, fov_height,
                                rank_background, consec_failures,
                                max_iters_main, max_iters_init, tol,
                                d_sub=d_sub, t_sub=t_sub)
background = np.asarray(batch_recompose(spatial_components_bg,
                                        temporal_components_bg,
                                        block_ranks,
                                        block_indices))
foreground = mov - background

In [ ]:
np.savez("endoscope_prepared.npz", mov, background, foreground)